In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Data
mice = np.array([
    [0, 0.56, 0, 0.44], 
    np.array([0.87, 0.01, 0, 0.08]) / 0.96, 
    np.array([0.56, 0.05, 0, 0.38]) / 0.99, 
    np.array([0.52, 0.06, 0.33, 0.08]) / 0.99, 
    [0.74, 0.07, 0.1, 0.09], 
    [0.69, 0.08, 0.16, 0.07], 
    [0.65, 0.07, 0.16, 0.12], 
    [0.56, 0.07, 0.29, 0.08], 
    np.array([0.56, 0.05, 0.27, 0.11]) / 0.99
])

Allbs = np.array([
    -1.1214, -0.760036, -0.498491, -1.47511, -1.38615, 
    -1.6076, -1.03975, -1.53805, -0.994873
])

# Parameters
tree_numbers = [20]
n_trials = 9

# Store results
results = {tn: [] for tn in tree_numbers}  # Store relative errors for each number of trees

# Loop over number of trees
for treenumber in tree_numbers:
    # Store trial predictions for each number of trees
    trial_predictions = []

    # 9 independent trials
    for _ in range(n_trials):
        trial_predictions_for_one_tree_count = []

        # Leave-one-out cross-validation
        for i in range(len(mice)):
            train_idx = list(set(range(len(mice))) - {i})  # Exclude one sample
            X_train, y_train = mice[train_idx], Allbs[train_idx]
            X_test, y_test = mice[i].reshape(1, -1), Allbs[i]

            # Train random forest
            model = RandomForestRegressor(n_estimators=treenumber, random_state=None)
            model.fit(X_train, y_train)

            # Predict
            y_pred = model.predict(X_test)[0]
            trial_predictions_for_one_tree_count.append(y_pred)

        # Store the predictions for this trial
        trial_predictions.append(trial_predictions_for_one_tree_count)

    # Compute the mean prediction for each trial (across samples)
    mean_predictions = np.mean(trial_predictions, axis=0)

    # Compute relative errors for the mean predictions
    relative_errors = np.abs((mean_predictions - Allbs) / Allbs)

    # Store the relative errors for this number of trees
    results[treenumber] = relative_errors

# Print results
for treenumber in tree_numbers:
    print(f"Relative errors for {treenumber} trees:")
    for i, error in enumerate(results[treenumber]):
        print(f"Host {i+1}: {error:.6f}")
    print(f"Mean relative error for {treenumber} trees: {np.mean(results[treenumber]):.6f}\n")
